## Initialization

In [3]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt 
from PIL import Image

In [4]:
# generally trainval takes 1 minute to initialize
dataroot = '/ocean/projects/cis220039p/cherieho/data/datasets/nuscenes_full'
nusc = NuScenes(version='v1.0-trainval', dataroot=dataroot, verbose=True)
# nusc.list_scenes()

Loading NuScenes tables for version v1.0-trainval...


23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 43.490 seconds.
Reverse indexing ...
Done reverse indexing in 8.4 seconds.


## Get list of scenes in Nuscenes data object in a given map

Based on https://github.com/nutonomy/nuscenes-devkit/blob/66d8f442b9df2631fd793a053b203e3d2d2d1590/python-sdk/nuscenes/map_expansion/map_api.py#L1202 (`render_egoposes_on_fancy_map()`)

and `render_egoposes_on_map()` https://github.com/nutonomy/nuscenes-devkit/blob/66d8f442b9df2631fd793a053b203e3d2d2d1590/python-sdk/nuscenes/nuscenes.py#L1783 

In [5]:
# Get list of logs in Nuscenes data object in a given map
log_location = 'singapore-onenorth'
log_tokens = [log['token'] for log in nusc.log if log['location'] == log_location]
print("There are {log_nums} logs in {log_loc}.".format(log_nums=len(log_tokens), log_loc=log_location))

# Filter scenes given logs.
scene_tokens_location = [e['token'] for e in nusc.scene if e['log_token'] in log_tokens]
print("There are {scene_nums} scenes in {log_loc}.".format(scene_nums=len(scene_tokens_location), log_loc=log_location))
print("Scenes are: ", [e['name'] for e in nusc.scene if e['token'] in scene_tokens_location])


# Ids of scenes with a bad match between localization and map.
scene_blacklist = [499, 515, 517] # all in boston-seaport

There are 23 logs in singapore-onenorth.
There are 183 scenes in singapore-onenorth.
Scenes are:  ['scene-0001', 'scene-0002', 'scene-0003', 'scene-0004', 'scene-0005', 'scene-0006', 'scene-0007', 'scene-0008', 'scene-0009', 'scene-0010', 'scene-0011', 'scene-0012', 'scene-0013', 'scene-0014', 'scene-0015', 'scene-0016', 'scene-0017', 'scene-0018', 'scene-0019', 'scene-0020', 'scene-0021', 'scene-0022', 'scene-0023', 'scene-0024', 'scene-0025', 'scene-0026', 'scene-0027', 'scene-0028', 'scene-0029', 'scene-0030', 'scene-0031', 'scene-0032', 'scene-0033', 'scene-0034', 'scene-0035', 'scene-0036', 'scene-0038', 'scene-0039', 'scene-0041', 'scene-0042', 'scene-0043', 'scene-0044', 'scene-0045', 'scene-0046', 'scene-0047', 'scene-0048', 'scene-0049', 'scene-0050', 'scene-0051', 'scene-0052', 'scene-0053', 'scene-0054', 'scene-0055', 'scene-0056', 'scene-0057', 'scene-0058', 'scene-0059', 'scene-0060', 'scene-0061', 'scene-0120', 'scene-0121', 'scene-0122', 'scene-0123', 'scene-0124', 'scen

## Get ego poses for each scene

In [10]:
map_poselist_dict = {}
poselist_dict = {}

print('Adding ego poses to map...')
for scene_token in tqdm(scene_tokens_location):

    # Get records from the database.
    scene_record = nusc.get('scene', scene_token)
    scene_name = scene_record['name']
    scene_id = int(scene_name.replace('scene-', ''))
    log_record = nusc.get('log', scene_record['log_token'])
    map_record = nusc.get('map', log_record['map_token'])
    map_mask = map_record['mask']

    # Print a warning if the localization is known to be bad.
    if scene_id in scene_blacklist:
        print('Warning: %s is known to have a bad fit between ego pose and map.' % scene_name)
    
    pose_list = []
    mappose_list = []


    # For each sample in the scene, store the ego pose.
    sample_tokens = nusc.field2token('sample', 'scene_token', scene_token)
    for sample_token in sample_tokens:
        sample_record = nusc.get('sample', sample_token)

        # Poses are associated with the sample_data. Here we use the lidar sample_data.
        sample_data_record = nusc.get('sample_data', sample_record['data']['LIDAR_TOP'])
        pose_record = nusc.get('ego_pose', sample_data_record['ego_pose_token'])
#         print(pose_record['translation'])

        pose_list.append(pose_record['translation'])
        mappose_list.append(np.concatenate(
            map_mask.to_pixel_coords(pose_record['translation'][0], pose_record['translation'][1])))
    
    # Add to dictionary
    poselist_dict[scene_token] = np.array(pose_list)
    map_poselist_dict[scene_token] = np.array(mappose_list)

#         # Calculate the pose on the map and append.
#         map_poses.append(np.concatenate(
#             map_mask.to_pixel_coords(pose_record['translation'][0], pose_record['translation'][1])))

Adding ego poses to map...


100%|██████████| 183/183 [00:04<00:00, 39.77it/s] 


In [2]:
# # Plot based on Nuscenes train val split
# from nuscenes.utils.splits import create_splits_scenes
# split_scene_list = create_splits_scenes(verbose=False)
# train_scene_list = split_scene_list['train']
# val_scene_list = split_scene_list['val']
# test_scene_list = split_scene_list['test']


# # Plot ego poses (each color is 1 scene)
# # TODO: split different colors for test
# for k, v in poselist_dict.items():
#     color = 'black'
#     scene_name = nusc.get('scene', k)['name']
#     if scene_name in train_scene_list:
#         # print(scene_name, 'train')
#         color = 'red'
#         linewidth = 1
#     elif scene_name in val_scene_list:
#         # print('val')
#         color = 'blue'
#         linewidth = 3
#     elif scene_name in test_scene_list:
#         # print('test')
#         color = 'green'
#         linewidth = 1
    
# #     print(v.shape)
#     # print(k)
#     plt.plot(v[:,0], v[:,1], color=color, linewidth=linewidth)
# # plt.legend()


# plt.title('Trajectories in {map_location}. \n Red:Train, Blue:Test'.format(map_location=log_location))
# plt.show()


# #TODO: plot on map 

In [3]:
# from nuscenes.utils.splits import create_splits_scenes
# split_scene_list = create_splits_scenes(verbose=False)
# print(split_scene_list.keys())

In [1]:
# Plot color based on if the scene overlaps with a test location

# Find which scenes overlap with test locations from pose_list_dict
test_location_point = [400, 1000]
# test_location_point = [1000, 750]
dist_threshold = 200
x_list = []
y_list = []
c_list = []
for k, v in poselist_dict.items():
    # Find if any point in list is within a given distance of test_location_point, use as test
    distance_list = np.linalg.norm(v[:,:2] - test_location_point, axis=1)
    min_distance_ind = np.argmin(distance_list)
    min_distance = distance_list[min_distance_ind]
    scene_name = nusc.get('scene', k)['name']
    
    # print(min_distance)
    # if scene_name == 'scene-0268':
    if min_distance < dist_threshold:
        color = 'blue'
        linewidth = 3
        # print('Scene {scene_name} is {dist}m away.'.format(scene_name=nusc.get('scene', k)['name'], dist_threshold=dist_threshold, dist=min_distance))
        # plt.text(v[0,0], v[0,1], nusc.get('scene', k)['name'], fontsize=10)
    else:
        color = 'red'
        linewidth = 0.25
    x_list += list(v[:,0])
    y_list += list(v[:,1])
    c_list += list([min_distance]) * len(v[:,0])
    # Plot scene traj
    plt.scatter(v[:,0], v[:,1], color=color, s=linewidth)
    # Plot where in scene traj is closest to test location 
    # plt.scatter(v[min_distance_ind,0], v[min_distance_ind,1], color='red', s=20) 
    # print("Closest point on scene traj is: {closest_point}. Distance is {min_distance}.".format(closest_point=v[min_distance_ind,:2], min_distance=min_distance))
    # print(np.linalg.norm(v[min_distance_ind,:2] - test_location_point, axis=0))

    # plt.scatter(v[:,0], v[:,1], s=1)
# print(x_list)
# plt.scatter(x_list, y_list, c=c_list, cmap='rainbow', s=1)


# Plot test location and ring
plt.scatter(test_location_point[0], test_location_point[1], color='green', s=100)
circle1 = plt.Circle((test_location_point[0], test_location_point[1]), dist_threshold, color='green', fill=False, label='Test Location')
plt.gca().add_patch(circle1)
plt.title('Trajectories in {map_location} within {dist_threshold}m of test location. \n Red:Train, Blue:Test'.format(map_location=log_location, dist_threshold=dist_threshold))
plt.show()
        




NameError: name 'poselist_dict' is not defined

In [33]:
np.linalg.norm(v[:,:2] - test_location_point, axis=1).shape

(40,)

In [13]:
print(nusc.get('sample_data', nusc.get('sample', nusc.scene[0]['first_sample_token'])['data']['CAM_FRONT']))
print(nusc.get('sample_data', nusc.get('sample', nusc.scene[0]['first_sample_token'])['data']['CAM_BACK']))

# nusc.get('sample_data', nusc.scene[0]['first_sample_token']['data']['CAM_FRONT'])

{'token': '020d7b4f858147558106c504f7f31bef', 'sample_token': 'e93e98b63d3b40209056d129dc53ceee', 'ego_pose_token': '020d7b4f858147558106c504f7f31bef', 'calibrated_sensor_token': '2e64b091b3b146a390c2606b9081343c', 'timestamp': 1531883530412470, 'fileformat': 'jpg', 'is_key_frame': True, 'height': 900, 'width': 1600, 'filename': 'samples/CAM_FRONT/n015-2018-07-18-11-07-57+0800__CAM_FRONT__1531883530412470.jpg', 'prev': '', 'next': 'caa2bfad0b8a4a8090cb0b803352cbc8', 'sensor_modality': 'camera', 'channel': 'CAM_FRONT'}
{'token': 'aab35aeccbda42de82b2ff5c278a0d48', 'sample_token': 'e93e98b63d3b40209056d129dc53ceee', 'ego_pose_token': 'aab35aeccbda42de82b2ff5c278a0d48', 'calibrated_sensor_token': '9452428b74ab49d590ae3d239b5aaf92', 'timestamp': 1531883530437525, 'fileformat': 'jpg', 'is_key_frame': True, 'height': 900, 'width': 1600, 'filename': 'samples/CAM_BACK/n015-2018-07-18-11-07-57+0800__CAM_BACK__1531883530437525.jpg', 'prev': '', 'next': '96ae32e59653448c9614b1cee365a3bd', 'sensor